## The goal is to create a model that helps determine which type of police units should be acting on event based on the time and location - is the crime violent?

In [1]:
import pandas as pd
import numpy as np

In [2]:
alldata=pd.read_csv('full.csv')

C:\Users\xe050991188\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
alldata.head()

,COMPNOS,NatureCode,INCIDENT_TYPE_DESCRIPTION,MAIN_CRIMECODE,REPTDISTRICT,REPORTINGAREA,FROMDATE,WEAPONTYPE,Shooting,DOMESTIC,SHIFT,Year,Month,DAY_WEEK,UCRPART,X,Y,STREETNAME,XSTREETNAME,Location
0,120420285.0,BERPTA,RESIDENTIAL BURGLARY,05RB,D4,629,07/08/2012 06:00:00 AM,Other,No,No,Last,2012,7,Sunday,Part One,763273.1791,2951498.962,ABERDEEN ST,NaN,"(42.34638135, -71.10379454)"
1,120419202.0,PSHOT,AGGRAVATED ASSAULT,04xx,B2,327,07/08/2012 06:03:00 AM,Firearm,Yes,No,Last,2012,7,Sunday,Part One,771223.1638,2940772.099,HOWARD AV,NaN,"(42.31684135, -71.07458456)"
2,120419213.0,ARMROB,ROBBERY,03xx,D4,625,07/08/2012 06:26:00 AM,Firearm,No,No,Last,2012,7,Sunday,Part One,765118.8605,2950217.536,JERSEY ST,QUEENSBERRY ST,"(42.34284135, -71.09698955)"
3,120419223.0,ALARMC,COMMERCIAL BURGLARY,05CB,B2,258,07/08/2012 06:56:00 AM,Other,No,No,Last,2012,7,Sunday,Part One,773591.8648,2940638.174,COLUMBIA RD,NaN,"(42.3164411, -71.06582908)"
4,120419236.0,ARMROB,ROBBERY,03xx,E18,496,07/08/2012 07:15:00 AM,Firearm,No,No,Last,2012,7,Sunday,Part One,759042.7315,2923832.681,COLLINS ST,NaN,"(42.27051636, -71.11989955)"


In [4]:
alldata.shape

(268056, 20)

In [5]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268056 entries, 0 to 268055
Data columns (total 20 columns):
COMPNOS                      255797 non-null float64
NatureCode                   266391 non-null object
INCIDENT_TYPE_DESCRIPTION    268056 non-null object
MAIN_CRIMECODE               268056 non-null object
REPTDISTRICT                 267699 non-null object
REPORTINGAREA                268056 non-null int64
FROMDATE                     268056 non-null object
WEAPONTYPE                   268056 non-null object
Shooting                     268056 non-null object
DOMESTIC                     268056 non-null object
SHIFT                        255791 non-null object
Year                         268056 non-null int64
Month                        268056 non-null int64
DAY_WEEK                     268056 non-null object
UCRPART                      268056 non-null object
X                            253075 non-null float64
Y                            253075 non-null float64
STREE

Drop COMPNOS and UCRPART as these are internal BDP report number, not useful for analysis. Dropping XSTREETNAME as it has way to many missing data and is not crucial for analysis.

In [6]:
alldata=alldata.drop(['COMPNOS','XSTREETNAME','UCRPART'],axis=1)

Dropping all rows with missing values, there is still over 240k rows left, all of them with full data. Through this operation over 24k rows are deleted, which is a little over 9% of data I began with.

In [37]:
alldata=alldata.dropna()
alldata.shape

(243547, 18)

In [8]:
alldata['INCIDENT_TYPE_DESCRIPTION'].unique()

array(['RESIDENTIAL BURGLARY', 'AGGRAVATED ASSAULT', 'ROBBERY',
       'COMMERCIAL BURGLARY', 'SIMPLE ASSAULT', 'MedAssist', 'BENoProp',
       'VAL', 'FRAUD', 'PropFound', 'InvPer', 'TOWED', 'WEAPONS CHARGE',
       'InvProp', 'DRUG CHARGES', 'PersLoc', 'PropLost', 'OTHER LARCENY',
       'AUTO THEFT', 'Arrest', 'VANDALISM', 'MVAcc',
       'CRIMES AGAINST CHILDREN', 'PhoneCalls',
       'LARCENY FROM MOTOR VEHICLE', 'FIRE', 'Ballist', 'PubDrink',
       'Argue', 'Restrain', 'Gather', 'Service', 'SkipFare', 'SearchWarr',
       '32GUN', 'Harass', 'EMBEZELLMENT', 'OTHER', 'PRISON', 'FORGERY',
       'PersMiss', 'DEATH INVESTIGATION', 'LICViol', 'Landlord',
       'Hazardous', 'Plates', '07RV', 'PropDam', 'SexReg', 'HOMICIDE',
       'OPERATING UNDER INFLUENCE', 'STOLEN PROPERTY CHARGES', 'Runaway',
       'DISORDERLY', 'HateCrim', 'PROSTITUTION CHARGES', 'TRESPASS',
       'ARSON', 'BurgTools', 'Explos', 'VIOLATION OF LIQUOR LAWS',
       'Harbor', 'InvVeh', 'Bomb', 'Aircraft', 'Labor'

In [9]:
alldata['REPTDISTRICT'].unique()

array(['D4', 'B2', 'E18', 'C11', 'A1', 'D14', 'C6', 'E13', 'B3', 'A7',
       'A15', 'E5'], dtype=object)

All Boston PD districts are represented according to http://bpdnews.com/districts/

In [10]:
alldata['WEAPONTYPE'].unique()

array(['Other', 'Firearm', 'Unarmed', 'Knife'], dtype=object)

In [11]:
alldata['SHIFT'].unique()

array(['Last', 'Day', 'First'], dtype=object)

In [12]:
alldata['NatureCode'].unique()

array(['BERPTA', 'PSHOT ', 'ARMROB', 'ALARMC', 'SHOT  ', 'THREAT',
       'REQP  ', 'ALARMI', 'BEIP  ', 'IVMV  ', 'LARCRT', 'FDPROP',
       'ILLPRK', 'INVEST', 'INJOFF', 'INVCHD', 'IVDRUG', 'ARREST',
       'MVANO ', 'ABRPT ', 'BERPT ', 'MISPER', 'UNK   ', 'LSTPRP',
       'STOLMV', 'BURN1 ', 'IVPER ', 'VANRPT', 'FIGHT ', 'PDRPT ',
       'IVPREM', 'BEMVRT', 'DVIP  ', 'FIRE  ', 'OVRDOS', 'ABIP  ',
       'DOGRPT', 'CD99  ', 'NIDV  ', 'HARPT ', 'PERGUN', 'ALARMH',
       'MVPED1', 'HITRUN', 'STOLEN', 'VIORDR', 'MISCRP', 'MVPED2',
       'UNCONS', 'EDP2  ', 'CARST ', 'HOMINV', 'CD9   ', 'PKNIFE',
       'SHOTS ', 'MEET  ', 'MISC  ', 'ABDWRP', 'DISTRB', 'VANIP ',
       'ASTEMS', 'TSTOP ', 'MVA   ', 'TRANSP', 'CD11  ', 'LARCIP',
       'MVAINJ', 'SHPLFT', 'FIREB ', 'TRAUMA', 'SDNDTH', 'CD35  ',
       'IVLIC ', 'ROBRPT', 'ROBIP ', 'TRACK ', 'FDWEAP', 'CD19  ',
       'SSTOP ', 'ALARMX', 'ABDWIP', 'MVBLDR', 'STAB  ', 'MVARPT',
       'CRIP  ', 'TRESPS', 'MVACC ', 'AOD   ', 'MSNCOM', 'FDCH

In [13]:
print (alldata['MAIN_CRIMECODE'].unique())
print (len(alldata['MAIN_CRIMECODE'].unique()))

['05RB' '04xx' '03xx' '05CB' '08xx' 'MedAssist' 'BENoProp' 'VAL' '11xx'
 'PropFound' 'InvPer' 'TOWED' '15xx' 'InvProp' '18xx' 'PersLoc' 'PropLost'
 '06xx' '07xx' 'Arrest' '14xx' 'MVAcc' '20xx' 'PhoneCalls' '06MV' 'FIRE'
 'Ballist' 'PubDrink' 'Argue' 'Restrain' 'Gather' 'Service' 'SkipFare'
 'SearchWarr' '32GUN' 'Harass' '12xx' 'OTHER' 'PRISON' '10xx' 'PersMiss'
 '01INV' 'LICViol' 'Landlord' 'Hazardous' 'Plates' '07RV' 'PropDam'
 'SexReg' '01xx' '21xx' '13xx' 'Runaway' '24xx' 'HateCrim' '16xx'
 'TRESPASS' '09xx' 'BurgTools' 'Explos' '22xx' 'Harbor' 'InvVeh' 'Bomb'
 'Aircraft' 'Labor' 'Manslaug' '19xx' 'BioThreat']
69


With 77 possible outcomes the range of categories is too broad for multiclass classification. 

The remaining possibility is to create a binary category of a violent crime. The division would be the WEAPONTYPE - if the criminal is armed the crime is supposed to be violent.

In [14]:
alldata.loc[(alldata['WEAPONTYPE']=='Unarmed'),'VIOLENT']=0
alldata.loc[(alldata['WEAPONTYPE']!='Unarmed')&(alldata['WEAPONTYPE']!='Personal Weapon (hand  foot  etc.)'),'VIOLENT']=1

In [15]:
pd.crosstab(index=alldata['WEAPONTYPE'],columns='sum')

col_0,sum
WEAPONTYPE,
Firearm,4917
Knife,3713
Other,49895
Unarmed,185022


In [16]:
pd.crosstab(index=alldata['VIOLENT'],columns='sum')

col_0,sum
VIOLENT,
0.0,185022
1.0,58525


In [17]:
alldata['FROMDATEDAY'] = pd.to_datetime(alldata['FROMDATE'], format='%m/%d/%Y %I:%M:%S %p')
alldata.set_index('FROMDATEDAY', inplace=True)
alldata.sort_index(inplace=True)

In [18]:
alldata['DAY'] = alldata.index.day
alldata['HOUR'] = alldata.index.hour

In [19]:
alldata.head()

,NatureCode,INCIDENT_TYPE_DESCRIPTION,MAIN_CRIMECODE,REPTDISTRICT,REPORTINGAREA,FROMDATE,WEAPONTYPE,Shooting,DOMESTIC,SHIFT,Year,Month,DAY_WEEK,X,Y,STREETNAME,Location,VIOLENT,DAY,HOUR
FROMDATEDAY,,,,,,,,,,,,,,,,,,,,
2012-07-08 06:00:00,BERPTA,RESIDENTIAL BURGLARY,05RB,D4,629,07/08/2012 06:00:00 AM,Other,No,No,Last,2012,7,Sunday,763273.1791,2951498.962,ABERDEEN ST,"(42.34638135, -71.10379454)",1.0,8,6
2012-07-08 06:03:00,PSHOT,AGGRAVATED ASSAULT,04xx,B2,327,07/08/2012 06:03:00 AM,Firearm,Yes,No,Last,2012,7,Sunday,771223.1638,2940772.099,HOWARD AV,"(42.31684135, -71.07458456)",1.0,8,6
2012-07-08 06:26:00,ARMROB,ROBBERY,03xx,D4,625,07/08/2012 06:26:00 AM,Firearm,No,No,Last,2012,7,Sunday,765118.8605,2950217.536,JERSEY ST,"(42.34284135, -71.09698955)",1.0,8,6
2012-07-08 06:56:00,ALARMC,COMMERCIAL BURGLARY,05CB,B2,258,07/08/2012 06:56:00 AM,Other,No,No,Last,2012,7,Sunday,773591.8648,2940638.174,COLUMBIA RD,"(42.3164411, -71.06582908)",1.0,8,6
2012-07-08 07:15:00,ARMROB,ROBBERY,03xx,E18,496,07/08/2012 07:15:00 AM,Firearm,No,No,Last,2012,7,Sunday,759042.7315,2923832.681,COLLINS ST,"(42.27051636, -71.11989955)",1.0,8,7


Dropping all columns that are not location or time related except crimecode that I want the model to train and predict

In [20]:
alldata.reset_index(drop=True,inplace=True)

In [21]:
alldata=alldata.drop(['MAIN_CRIMECODE','NatureCode','INCIDENT_TYPE_DESCRIPTION','FROMDATE','WEAPONTYPE','Shooting','DOMESTIC','X','Y','Location'],axis=1)

Preprocessing features with Label Encoder for category that I want to predict and get dummies for binarized data of time related values.

In [22]:
from sklearn.preprocessing import LabelEncoder
cols = ('REPTDISTRICT', 'REPORTINGAREA', 'STREETNAME', )

for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(alldata[c].values)) 
    alldata[c] = lbl.transform(list(alldata[c].values))

In [23]:
alldata = pd.get_dummies(alldata)

In [24]:
alldata.head()

,REPTDISTRICT,REPORTINGAREA,Year,Month,STREETNAME,VIOLENT,DAY,HOUR,SHIFT_Day,SHIFT_First,SHIFT_Last,DAY_WEEK_Friday,DAY_WEEK_Monday,DAY_WEEK_Saturday,DAY_WEEK_Sunday,DAY_WEEK_Thursday,DAY_WEEK_Tuesday,DAY_WEEK_Wednesday
0,8,627,2012,7,19,1.0,8,6,0,0,1,0,0,0,1,0,0,0
1,3,325,2012,7,1837,1.0,8,6,0,0,1,0,0,0,1,0,0,0
2,8,623,2012,7,1921,1.0,8,6,0,0,1,0,0,0,1,0,0,0
3,3,256,2012,7,825,1.0,8,6,0,0,1,0,0,0,1,0,0,0
4,10,494,2012,7,819,1.0,8,7,0,0,1,0,0,0,1,0,0,0


Taking a sample of the data (30%) to choose best parameters with GridSearch CV

In [25]:
sampledata=alldata.sample(frac=0.3,random_state=18)
sampledata.shape

(73064, 18)

In [27]:
from sklearn.model_selection import train_test_split

X_all = sampledata.drop(['VIOLENT'], axis=1)
y_all = sampledata['VIOLENT']

num_test = 0.30
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=18)

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

clfRF = RandomForestClassifier()

parameters = {'n_estimators': [6, 9], 
              'max_features': ['log2', 'sqrt','auto'], 
              'criterion': ['entropy', 'gini'],
              'max_depth': [5, 10], 
              'min_samples_split': [3, 5],
              'min_samples_leaf': [5,8]
             }

acc_scorer = make_scorer(accuracy_score)

grid_obj = GridSearchCV(clfRF, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

clfRF = grid_obj.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='log2', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=8,
            min_samples_split=5, min_weight_fraction_leaf=0.0,
            n_estimators=9, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [29]:
from sklearn.neighbors import KNeighborsClassifier
clfKN = KNeighborsClassifier()

parameters = {'n_neighbors': [3,5,7,9], 
             }

acc_scorer = make_scorer(accuracy_score)

grid_obj = GridSearchCV(clfKN, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

clfKN = grid_obj.best_estimator_

clfKN.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=9, p=2,
           weights='uniform')

In [35]:
from sklearn.svm import LinearSVC

clfSV = LinearSVC()

parameters = {'C': [0.01, 0.1, 1]
             }

acc_scorer = make_scorer(accuracy_score)

grid_obj = GridSearchCV(clfSV, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

clfSV = grid_obj.best_estimator_

clfSV.fit(X_train, y_train)

LinearSVC(C=0.01, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [30]:
X_alldata = alldata.drop(['VIOLENT'], axis=1)
y_alldata = alldata['VIOLENT']

num_test = 0.30
X_trainall, X_testall, y_trainall, y_testall = train_test_split(X_alldata, y_alldata, test_size=num_test, random_state=18)

In [31]:
clfRF.fit(X_trainall, y_trainall)
predictionsRF = clfRF.predict(X_testall)
print(accuracy_score(y_testall, predictionsRF))

0.760473550948


In [33]:
clfKN.fit(X_trainall, y_trainall)
predictionsKN = clfKN.predict(X_testall)
print(accuracy_score(y_testall, predictionsKN))

0.739191131185


In [36]:
clfSV.fit(X_trainall, y_trainall)
predictionsSV = clfSV.predict(X_testall)
print(accuracy_score(y_testall, predictionsSV))

0.760090330528


The best algorithm form ones tested is RandomForest. Now it should be checked with K-Fold for overfitting.

In [38]:
from sklearn.cross_validation import KFold

def run_kfold(clf):
    kf = KFold(243547, n_folds=5)
    outcomes = []
    fold = 0
    for train_index, test_index in kf:
        fold += 1
        X_trainall, X_testall = X_alldata.values[train_index], X_alldata.values[test_index]
        y_trainall, y_testall = y_alldata.values[train_index], y_alldata.values[test_index]
        clfRF.fit(X_trainall, y_trainall)
        predictionsK = clfRF.predict(X_testall)
        accuracy = accuracy_score(y_testall, predictionsK)
        outcomes.append(accuracy)
        print("Fold {0} accuracy: {1}".format(fold, accuracy))     
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 

run_kfold(clfRF)

C:\Users\xe050991188\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Fold 1 accuracy: 0.7377335249435434
Fold 2 accuracy: 0.7530486553069184
Fold 3 accuracy: 0.7665934426902625
Fold 4 accuracy: 0.7664907922560512
Fold 5 accuracy: 0.7763247038534973
Mean Accuracy: 0.7600382238100545


Cross-validation doesn't indicate overfitting, the model should give about 76% accuracy. 